In [1]:
#Create Folder for modelling checkpoint
import os
import pandas as pd
# pickle to save dictionary in file
import pickle 
cwd = os.path.normpath(os.getcwd()+ os.sep + os.pardir + os.sep + os.pardir)

In [3]:
def loadResultDictionary(cwd, dictinaryName):
    with open(cwd + f'/results/{dictinaryName}.pkl', 'rb') as f:
        dictionary = pickle.load(f)
    return dictionary

def getMetricsForClientInCluster(dictionary, model, horizon, feature, cluster, client, mse, rmse, mape, mae, time): 
    mse = mse + dictionary['Federated'][model][horizon][feature][cluster][client]['MSE']
    rmse = rmse+ dictionary['Federated'][model][horizon][feature][cluster][client]['RMSE']
    mape = mape + dictionary['Federated'][model][horizon][feature][cluster][client]['MAPE']
    mae = mae + dictionary['Federated'][model][horizon][feature][cluster][client]['MAE']
    time = time + dictionary['Federated'][model][horizon][feature][cluster][client]['Time']
    
    return mse, rmse, mape, mae, time

def getMetricsForClientInLocal(dictionary, model, horizon, feature, client, mse, rmse, mape, mae, time): 
    mse = mse + dictionary['Local'][model][horizon][feature][client]['MSE']
    rmse = rmse+ dictionary['Local'][model][horizon][feature][client]['RMSE']
    mape = mape + dictionary['Local'][model][horizon][feature][client]['MAPE']
    mae = mae + dictionary['Local'][model][horizon][feature][client]['MAE']
    time = time + dictionary['Local'][model][horizon][feature][client]['Time']
    
    return mse, rmse, mape, mae, time

def getMetricsForCentral(dictionary, model, horizon, feature, mse, rmse, mape, mae, time): 
    mse = mse + dictionary['Central'][model][horizon][feature]['MSE']
    rmse = rmse+ dictionary['Central'][model][horizon][feature]['RMSE']
    mape = mape + dictionary['Central'][model][horizon][feature]['MAPE']
    mae = mae + dictionary['Central'][model][horizon][feature]['MAE']
    time = time + dictionary['Central'][model][horizon][feature]['Time']
    
    return mse, rmse, mape, mae, time

def getMetricsForCluster(cluster_length, mse, rmse, mape, mae, time):
    mse = mse/cluster_length
    rmse = rmse /cluster_length
    mape = mape/cluster_length
    mae = mae/cluster_length
    time = time/cluster_length
    return mse, rmse, mape, mae, time

def namestr(obj, namespace):
    """
    Get the name of the variable as a String
    """
    return [name for name in namespace if namespace[name] is obj][0]

def getResultsFromFederatedDictionary(dictionaries): 
    #Create dataframe with Coumns(Szenario, MSE, RMSE, MAPE, MAE, Time)
    result_df = pd.DataFrame(columns=['Learning', 'Horizon', 'Feature', 'Model', 'MSE', 'RMSE', 'MAPE', 'MAE', 'Time'])
    
    for dictionary in dictionaries:
        horizon = namestr(dictionary, globals()).split('_')[1]
        feature = namestr(dictionary, globals()).split('_')[2]

        # For each of the models LSTM, CNN, Transformer
        for model in dictionary['Federated']:
            
            #initilize cluster_length and metrics
            mse, rmse, mape, mae, time = 0,0,0,0,0
            #For each of the clusters 0-5
            for cluster in dictionary['Federated'][model][horizon][feature]:
                #print("Model: ", model, "h: ", horizon, "f: ", feature)
                
                #Get the metrics for each client and add them up
                for idx, client in enumerate(dictionary['Federated'][model][horizon][feature][cluster]):
                    mse, rmse, mape, mae, time = getMetricsForClientInCluster(
                        dictionary, model, horizon, feature,
                        cluster, client, mse, rmse, mape, mae, time
                    )
                    
            #Devide the metrics by the number of clients in cluster to get average
            mse, rmse, mape, mae, time = getMetricsForCluster(33, mse, rmse, mape, mae, time)
            #Add metrics to the last row of the result_df
            result_df.loc[len(result_df)]= ['Federated', horizon, feature, model, mse, rmse, mape, mae, time]
            
    return result_df

def getResultsFromCentralDictionary(dictionaries): 
    #Create dataframe with Coumns(Szenario, MSE, RMSE, MAPE, MAE, Time)
    result_df = pd.DataFrame(columns=['Learning', 'Horizon', 'Feature', 'Model', 'MSE', 'RMSE', 'MAPE', 'MAE', 'Time'])
    
    for dictionary in dictionaries:
        horizon = namestr(dictionary, globals()).split('_')[1]
        feature = namestr(dictionary, globals()).split('_')[2]

        # For each of the models LSTM, CNN, Transformer
        for model in dictionary['Central']:

            mse, rmse, mape, mae, time = 0,0,0,0,0

            #Get the metrics for each client and add them up
            for idx, client in enumerate(dictionary['Central'][model][horizon][feature]):
                mse, rmse, mape, mae, time = getMetricsForCentral(
                    dictionary, model, horizon, feature,
                    mse, rmse, mape, mae, time
                )
           
            #Add metrics to the last row of the result_df
            result_df.loc[len(result_df)]= ['Central', horizon, feature, model, mse, rmse, mape, mae, time]

    return result_df


def getResultsFromLocalDictionary(dictionaries): 
    #Create dataframe with Coumns(Szenario, MSE, RMSE, MAPE, MAE, Time)
    result_df = pd.DataFrame(columns=['Learning', 'Horizon', 'Feature', 'Model', 'MSE', 'RMSE', 'MAPE', 'MAE', 'Time'])
    
    for dictionary in dictionaries:
        horizon = namestr(dictionary, globals()).split('_')[1]
        feature = namestr(dictionary, globals()).split('_')[2]

        # For each of the models LSTM, CNN, Transformer
        for model in dictionary['Local']:

            #initilize cluster_length and metrics
            mse, rmse, mape, mae, time = 0,0,0,0,0

            #Get the metrics for each client and add them up
            for idx, client in enumerate(dictionary['Local'][model][horizon][feature]):

                mse, rmse, mape, mae, time = getMetricsForClientInLocal(
                    dictionary, model, horizon, feature,
                    client, mse, rmse, mape, mae, time
                )
                
            #Devide the metrics by the number of clients in cluster to get average
            mse, rmse, mape, mae, time = getMetricsForCluster(33, mse, rmse, mape, mae, time)
            #Add metrics to the last row of the result_df
            result_df.loc[len(result_df)]= ['Local', horizon, feature, model, mse, rmse, mape, mae, time]

    return result_df

In [4]:
fl_H12_F7 = loadResultDictionary(cwd, 'Federated_results_H12_F7')
dictionaries = [fl_H12_F7]

result_df = pd.DataFrame(columns=['Learning', 'Horizon', 'Feature', 'Model', 'Cluster', 'MSE', 'RMSE', 'MAPE', 'MAE', 'Time'])  

for dictionary in dictionaries:
    horizon = namestr(dictionary, globals()).split('_')[1]
    feature = namestr(dictionary, globals()).split('_')[2]
    
    for model in dictionary['Federated']:

        #initilize cluster_length and metrics
        mse, rmse, mape, mae, time = 0,0,0,0,0
        #For each of the clusters 0-5
        for cluster in dictionary['Federated'][model][horizon][feature]:
            #print("Model: ", model, "h: ", horizon, "f: ", feature)
            cluster_length = 0
            #Get the metrics for each client and add them up
            for idx, client in enumerate(dictionary['Federated'][model][horizon][feature][cluster]):
                mse, rmse, mape, mae, time = getMetricsForClientInCluster(
                    dictionary, model, horizon, feature,
                    cluster, client, mse, rmse, mape, mae, time
                )
                cluster_length = cluster_length + 1
            #Devide the metrics by the number of clients in cluster to get average
            mse, rmse, mape, mae, time = getMetricsForCluster(cluster_length, mse, rmse, mape, mae, time)
            #print(cluster, " : ",mse, rmse, mape, mae, time)
            result_df.loc[len(result_df)]= ['Federated', horizon, feature, model, cluster, mse, rmse, mape, mae, time]

df_f_results_f = result_df.groupby(['Horizon','Feature', 'Model', 'Cluster'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()
df_f_results_f

C:\Users\jonas\AppData\Local\Temp\ipykernel_12104\3769743233.py:30: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_f_results_f = result_df.groupby(['Horizon','Feature', 'Model', 'Cluster'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()


MSE      RMSE          MAPE  \
Horizon Feature Model       Cluster                                     
H12     F7      CNN         0        0.047317  0.216986  1.041335e+08   
                            1        0.034728  0.177353  7.631907e+07   
                            2        0.032166  0.192197  8.928659e+07   
                            3        0.072083  0.285891  1.376293e+08   
                            4        0.095153  0.385280  1.899015e+08   
                            5        0.084671  0.308002  2.066234e+08   
                LSTM        0        0.041943  0.204211  8.718837e+07   
                            1        0.025737  0.155474  6.168967e+07   
                            2        0.022817  0.158513  5.870843e+07   
                            3        0.058903  0.256536  1.131674e+08   
                            4        0.056808  0.293433  1.364711e+08   
                            5        0.030460  0.185401  8.015408e+07   
                Transformer 0        0.041735  0.203681  8.899351e+07   
                            1        0.025351  0.153212  5.853470e+07   
                            2        0.022734  0.157981  5.964697e+07   
                            3        0.058678  0.256055  1.129984e+08   
                            4        0.056759  0.293457  1.322288e+08   
                            5        0.031059  0.186594  8.579866e+07   

                                          MAE      Time  
Horizon Feature Model       Cluster                      
H12     F7      CNN         0        0.196263  0.277498  
                            1        0.127890  0.352295  
                            2        0.155981  0.414157  
                            3        0.252430  0.428886  
                            4        0.326565  0.561990  
                            5        0.265775  0.438993  
                LSTM        0        0.183136  2.647967  
                            1        0.110927  7.920683  
                            2        0.117560  8.837963  
                            3        0.227137  6.341570  
                            4        0.242925  9.632616  
                            5        0.130379  9.300850  
                Transformer 0        0.183224  2.093326  
                            1        0.108049  3.127701  
                            2        0.121443  3.590000  
                            3        0.226982  3.553251  
                            4        0.242611  5.689381  
                            5        0.136559  4.367039

In [7]:
df_f_results_l

MSE      RMSE          MAPE  \
Horizon Feature Model       Cluster                                     
H12     F7      CNN         0        0.045735  0.213331  9.862112e+07   
                            1        0.032407  0.155210  7.920106e+07   
                            2        0.026098  0.159471  7.665492e+07   
                            3        0.064059  0.251633  1.199477e+08   
                            4        0.036414  0.190824  9.213020e+07   
                            5        0.073914  0.261067  1.878516e+08   
                LSTM        0        0.045653  0.212982  9.647504e+07   
                            1        0.026796  0.141823  6.541274e+07   
                            2        0.025202  0.157486  7.791406e+07   
                            3        0.062044  0.247887  1.228248e+08   
                            4        0.042074  0.205115  9.653212e+07   
                            5        0.053225  0.224708  1.608736e+08   
                Transformer 0        0.041725  0.203689  8.760219e+07   
                            1        0.019952  0.125917  4.508975e+07   
                            2        0.018158  0.128468  4.499278e+07   
                            3        0.056132  0.234994  1.008412e+08   
                            4        0.027590  0.165979  7.838949e+07   
                            5        0.024615  0.152621  6.528325e+07   

                                          MAE      Time  
Horizon Feature Model       Cluster                      
H12     F7      CNN         0        0.192989  0.407037  
                            1        0.114422  0.392127  
                            2        0.132841  0.329317  
                            3        0.223885  0.409438  
                            4        0.161432  0.483067  
                            5        0.227574  0.354834  
                LSTM        0        0.195561  9.956258  
                            1        0.102147  6.256895  
                            2        0.131830  5.321476  
                            3        0.226833  8.297066  
                            4        0.179395  6.981588  
                            5        0.204633  7.450014  
                Transformer 0        0.182799  2.136334  
                            1        0.081813  2.055001  
                            2        0.099941  2.499778  
                            3        0.210581  4.158641  
                            4        0.129257  1.605001  
                            5        0.103921  3.493188

In [6]:
#Local H12_F7
y=[2, 2, 4, 4, 1, 5, 1, 5, 1, 5, 5, 5, 5, 1, 3, 0, 3, 0, 5, 1, 3, 3, 3, 3, 5, 2, 2, 1, 3, 0, 0, 1, 2]
df = pd.read_csv(cwd+'/data/d03_data_processed/d03_data_processed.csv', encoding= 'unicode_escape', index_col='Date')
#Display smart meter names and amount
smart_meter_names = df.columns[2:-4]

l_H12_F7 = loadResultDictionary(cwd, 'Local_results_H12_F7')

#Add cluster to the client
for idx, client in enumerate(smart_meter_names):
    # Füge die clienten ein, die zu dem Cluster gehören
    l_H12_F7['Local']['LSTM']['H12']['F7'][client]['Cluster'] = y[idx]
    l_H12_F7['Local']['CNN']['H12']['F7'][client]['Cluster'] = y[idx]
    l_H12_F7['Local']['Transformer']['H12']['F7'][client]['Cluster'] = y[idx]
    
result_df = pd.DataFrame(columns=['Learning', 'Horizon', 'Feature', 'Model', 'Cluster', 'MSE', 'RMSE', 'MAPE', 'MAE', 'Time'])

horizon = 'H12'
feature = 'F7'

# For each of the models LSTM, CNN, Transformer
for model in l_H12_F7['Local']:

    #initilize cluster_length and metrics
    mse, rmse, mape, mae, time = 0,0,0,0,0

    #Get the metrics for each client and add them up
    for idx, client in enumerate(l_H12_F7['Local'][model][horizon][feature]):
        mse = l_H12_F7['Local'][model][horizon][feature][client]['MSE']
        rmse = l_H12_F7['Local'][model][horizon][feature][client]['RMSE']
        mape = l_H12_F7['Local'][model][horizon][feature][client]['MAPE']
        mae = l_H12_F7['Local'][model][horizon][feature][client]['MAE']
        time = l_H12_F7['Local'][model][horizon][feature][client]['Time']
        cluster = l_H12_F7['Local'][model][horizon][feature][client]['Cluster']
        #Add metrics to the last row of the result_df
        result_df.loc[len(result_df)]= ['Local', horizon, feature, model, cluster, mse, rmse, mape, mae, time]

df_f_results_l = result_df.groupby(['Horizon','Feature', 'Model', 'Cluster'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].mean()
df_f_results_l


C:\Users\jonas\AppData\Local\Temp\ipykernel_12104\3664909328.py:38: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_f_results_l = result_df.groupby(['Horizon','Feature', 'Model', 'Cluster'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].mean()


MSE      RMSE          MAPE  \
Horizon Feature Model       Cluster                                     
H12     F7      CNN         0        0.045735  0.213331  9.862112e+07   
                            1        0.032407  0.155210  7.920106e+07   
                            2        0.026098  0.159471  7.665492e+07   
                            3        0.064059  0.251633  1.199477e+08   
                            4        0.036414  0.190824  9.213020e+07   
                            5        0.073914  0.261067  1.878516e+08   
                LSTM        0        0.045653  0.212982  9.647504e+07   
                            1        0.026796  0.141823  6.541274e+07   
                            2        0.025202  0.157486  7.791406e+07   
                            3        0.062044  0.247887  1.228248e+08   
                            4        0.042074  0.205115  9.653212e+07   
                            5        0.053225  0.224708  1.608736e+08   
                Transformer 0        0.041725  0.203689  8.760219e+07   
                            1        0.019952  0.125917  4.508975e+07   
                            2        0.018158  0.128468  4.499278e+07   
                            3        0.056132  0.234994  1.008412e+08   
                            4        0.027590  0.165979  7.838949e+07   
                            5        0.024615  0.152621  6.528325e+07   

                                          MAE      Time  
Horizon Feature Model       Cluster                      
H12     F7      CNN         0        0.192989  0.407037  
                            1        0.114422  0.392127  
                            2        0.132841  0.329317  
                            3        0.223885  0.409438  
                            4        0.161432  0.483067  
                            5        0.227574  0.354834  
                LSTM        0        0.195561  9.956258  
                            1        0.102147  6.256895  
                            2        0.131830  5.321476  
                            3        0.226833  8.297066  
                            4        0.179395  6.981588  
                            5        0.204633  7.450014  
                Transformer 0        0.182799  2.136334  
                            1        0.081813  2.055001  
                            2        0.099941  2.499778  
                            3        0.210581  4.158641  
                            4        0.129257  1.605001  
                            5        0.103921  3.493188

In [8]:
#Load all 4 federated results dictionaries
fl_H12_F7 = loadResultDictionary(cwd, 'Federated_results_H12_F7')
fl_H24_F7 = loadResultDictionary(cwd, 'Federated_results_H24_F7')
fl_H12_F5 = loadResultDictionary(cwd, 'Federated_results_H12_F5')
fl_H24_F5 = loadResultDictionary(cwd, 'Federated_results_H24_F5')

df_f = getResultsFromFederatedDictionary([fl_H12_F7, fl_H24_F7, fl_H12_F5, fl_H24_F5])

df_f_results = df_f.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()
df_f_results

C:\Users\jonas\AppData\Local\Temp\ipykernel_17632\1292235094.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_f_results = df_f.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()


MSE      RMSE          MAPE       MAE  \
Horizon Feature Model                                                     
H12     F5      CNN          0.049176  0.211138  1.082899e+08  0.177732   
                LSTM         0.031181  0.166285  7.064704e+07  0.130924   
                Transformer  0.030974  0.165930  7.074782e+07  0.131556   
        F7      CNN          0.051030  0.213591  1.150341e+08  0.180224   
                LSTM         0.031073  0.166527  7.040136e+07  0.131333   
                Transformer  0.031077  0.166205  7.131471e+07  0.132749   
H24     F5      CNN          0.050357  0.212255  1.097430e+08  0.179420   
                LSTM         0.032346  0.170133  7.557743e+07  0.137586   
                Transformer  0.032657  0.171160  7.748699e+07  0.140476   
        F7      CNN          0.048485  0.209480  1.095266e+08  0.177795   
                LSTM         0.033312  0.173044  7.819700e+07  0.140607   
                Transformer  0.032728  0.171358  7.574309e+07  0.139258   

                                 Time  
Horizon Feature Model                  
H12     F5      CNN          0.544285  
                LSTM         8.121141  
                Transformer  2.587565  
        F7      CNN          0.340913  
                LSTM         6.451778  
                Transformer  3.071254  
H24     F5      CNN          0.650531  
                LSTM         8.778090  
                Transformer  5.059290  
        F7      CNN          0.464844  
                LSTM         7.891770  
                Transformer  3.799042

In [9]:
#Load all 4 federated results dictionaries
c_H12_F7 = loadResultDictionary(cwd, 'Central_results_H12_F7')
c_H24_F7 = loadResultDictionary(cwd, 'Central_results_H24_F7')
c_H12_F5 = loadResultDictionary(cwd, 'Central_results_H12_F5')
c_H24_F5 = loadResultDictionary(cwd, 'Central_results_H24_F5')

df_c = getResultsFromCentralDictionary([c_H12_F7, c_H24_F7, c_H12_F5, c_H24_F5])
df_c_results = df_c.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()
df_c_results

C:\Users\jonas\AppData\Local\Temp\ipykernel_17632\1490845994.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_c_results = df_c.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()


MSE      RMSE          MAPE       MAE  \
Horizon Feature Model                                                     
H12     F5      CNN          0.488511  1.562867  1.099057e+09  1.401436   
                LSTM         0.470900  1.534437  1.140369e+09  1.405556   
                Transformer  0.206473  1.016054  6.208793e+08  0.888174   
        F7      CNN          0.657494  1.813138  1.341811e+09  1.626052   
                LSTM         0.470904  1.534443  1.140370e+09  1.405557   
                Transformer  0.201017  1.002538  6.111885e+08  0.884497   
H24     F5      CNN          0.456040  1.510034  1.084901e+09  1.368384   
                LSTM         0.472595  1.537200  1.142905e+09  1.407908   
                Transformer  0.216681  1.040867  6.495527e+08  0.933119   
        F7      CNN          0.415402  1.441183  1.016125e+09  1.311138   
                LSTM         0.472597  1.537205  1.142907e+09  1.407910   
                Transformer  0.208946  1.022120  6.163708e+08  0.906014   

                                   Time  
Horizon Feature Model                    
H12     F5      CNN           13.318900  
                LSTM         508.136090  
                Transformer  385.457190  
        F7      CNN           51.337567  
                LSTM         750.632808  
                Transformer  320.880664  
H24     F5      CNN           26.636164  
                LSTM         495.053867  
                Transformer  513.301120  
        F7      CNN           32.737389  
                LSTM         793.414520  
                Transformer  213.256256

In [10]:
#Load all 4 federated results dictionaries
l_H12_F7 = loadResultDictionary(cwd, 'Local_results_H12_F7')
l_H24_F7 = loadResultDictionary(cwd, 'Local_results_H24_F7')
l_H12_F5 = loadResultDictionary(cwd, 'Local_results_H12_F5')
l_H24_F5 = loadResultDictionary(cwd, 'Local_results_H24_F5')

df_l = getResultsFromLocalDictionary([l_H12_F5, l_H24_F5, l_H12_F7, l_H24_F7])
df_l_results = df_l.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()
df_l_results

C:\Users\jonas\AppData\Local\Temp\ipykernel_17632\3859397800.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_l_results = df_l.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()


MSE      RMSE          MAPE       MAE  \
Horizon Feature Model                                                     
H12     F5      CNN          0.049431  0.209431  1.116804e+08  0.178732   
                LSTM         0.042449  0.195009  1.045926e+08  0.167265   
                Transformer  0.031565  0.168021  7.239336e+07  0.134788   
        F7      CNN          0.050086  0.211175  1.169356e+08  0.180235   
                LSTM         0.043650  0.199249  1.082784e+08  0.173943   
                Transformer  0.031587  0.167770  6.896771e+07  0.132350   
H24     F5      CNN          0.047891  0.208044  1.119704e+08  0.178646   
                LSTM         0.044160  0.199872  1.090453e+08  0.174171   
                Transformer  0.032448  0.170556  7.656419e+07  0.139227   
        F7      CNN          0.048543  0.209579  1.114164e+08  0.178187   
                LSTM         0.043309  0.198418  1.078646e+08  0.172843   
                Transformer  0.032626  0.171401  7.482899e+07  0.138459   

                                 Time  
Horizon Feature Model                  
H12     F5      CNN          0.451383  
                LSTM         6.660658  
                Transformer  2.262566  
        F7      CNN          0.384561  
                LSTM         7.329498  
                Transformer  2.899856  
H24     F5      CNN          0.459682  
                LSTM         8.462139  
                Transformer  2.760343  
        F7      CNN          0.552470  
                LSTM         9.623751  
                Transformer  3.634432